#### This script predicts variables (like stomatal conductance, LE, T, etc.) using trained ipure ML model of stomatal conductnace under Configuration-1.

In [1]:
import numpy as np
# SW_model_LE: calculate the LE using Shutteleworth-Wallace model given rsc and other input variable
def SW_model_LE_np(auxi, rsc_pre):
    
    #---- SW-Model ----
    # --> LE = LE_e + LE_c = w_s*PM_s + w_c*PM_c
    # --> PM_s = (delta*A + (rho*Cp*VPD - delta*ras*(A - A_s))/(raa + ras))/(delta + Psy*(1 + rss/(raa + ras)))
    # --> PM_c = (delta*A + (rho*Cp*VPD - delta*rac*A_s)/(raa + rac))/(delta + Psy*(1 + rsc/(raa + rac)))
    # --> w_s = 1/(1 + R_s*R_a/(R_c*(R_s + R_a)))
    # --> w_c = 1/(1 + R_c*R_a/(R_s*(R_c + R_a)))
    # --> R_s = (delta + Psy)*ras + Psy*rss
    # --> R_c = (delta + Psy)*rac + Psy*rsc
    # --> R_a = (delta + Psy)*raa
    
    # --> A = Rn - G
    # --> A_s = Rns - G
    # --> Rns = Rn*exp(-Kr*LAI)
    #----- END ----

    Kr = 0.6
    A = auxi[25,:] - auxi[26,:]
    Rns = auxi[25,:]*np.exp(-Kr*auxi[27,:])
    A_s = Rns - auxi[26,:]
    delta = auxi[17,:]
    rho = auxi[18,:]
    Cp = auxi[20,:]
    Psy = auxi[19,:]
    VPD = auxi[28,:]/10   # Check the units (should be in kPa)
    ras = auxi[21,:]
    rac = auxi[22,:]
    raa = auxi[23,:]
    rss = auxi[16,:]
    PM_s = (delta*A + (rho*Cp*VPD - delta*ras*(A - A_s))/(raa + ras))/(delta + Psy*(1 + rss/(raa + ras)))
    rsc = np.exp(rsc_pre)  # y = log(rsc)
    rsc = np.clip(rsc,1e-5,5000)
    PM_c = (delta*A + (rho*Cp*VPD - delta*rac*A_s)/(raa + rac))/(delta + Psy*(1 + rsc/(raa + rac)))
    R_s = (delta + Psy)*ras + Psy*rss
    R_c = (delta + Psy)*rac + Psy*rsc
    R_a = (delta + Psy)*raa
    w_s = 1/(1 + R_s*R_a/(R_c*(R_s + R_a)))
    w_c = 1/(1 + R_c*R_a/(R_s*(R_c + R_a)))
    
    T_pre = w_c*PM_c
    LE_pre = w_s*PM_s + w_c*PM_c
    
    return LE_pre

# SW_model_T: calculate the T (Transpiration) using Shutteleworth-Wallace model given rsc and other input variable
def SW_model_T_np(auxi, rsc_pre):
    
    #---- SW-Model ----
    # --> LE = LE_e + LE_c = w_s*PM_s + w_c*PM_c
    # --> PM_s = (delta*A + (rho*Cp*VPD - delta*ras*(A - A_s))/(raa + ras))/(delta + Psy*(1 + rss/(raa + ras)))
    # --> PM_c = (delta*A + (rho*Cp*VPD - delta*rac*A_s)/(raa + rac))/(delta + Psy*(1 + rsc/(raa + rac)))
    # --> w_s = 1/(1 + R_s*R_a/(R_c*(R_s + R_a)))
    # --> w_c = 1/(1 + R_c*R_a/(R_s*(R_c + R_a)))
    # --> R_s = (delta + Psy)*ras + Psy*rss
    # --> R_c = (delta + Psy)*rac + Psy*rsc
    # --> R_a = (delta + Psy)*raa
    
    # --> A = Rn - G
    # --> A_s = Rns - G
    # --> Rns = Rn*exp(-Kr*LAI)
    #----- END ----

    Kr = 0.6
    A = auxi[25,:] - auxi[26,:]
    Rns = auxi[25,:]*np.exp(-Kr*auxi[27,:])
    A_s = Rns - auxi[26,:]
    delta = auxi[17,:]
    rho = auxi[18,:]
    Cp = auxi[20,:]
    Psy = auxi[19,:]
    VPD = auxi[28,:]/10   # Check the units (should be in kPa)
    ras = auxi[21,:]
    rac = auxi[22,:]
    raa = auxi[23,:]
    rss = auxi[16,:]
    PM_s = (delta*A + (rho*Cp*VPD - delta*ras*(A - A_s))/(raa + ras))/(delta + Psy*(1 + rss/(raa + ras)))
    rsc = np.exp(rsc_pre)  # y = log(rsc)
    rsc = np.clip(rsc,1e-5,5000)
    PM_c = (delta*A + (rho*Cp*VPD - delta*rac*A_s)/(raa + rac))/(delta + Psy*(1 + rsc/(raa + rac)))
    R_s = (delta + Psy)*ras + Psy*rss
    R_c = (delta + Psy)*rac + Psy*rsc
    R_a = (delta + Psy)*raa
    w_s = 1/(1 + R_s*R_a/(R_c*(R_s + R_a)))
    w_c = 1/(1 + R_c*R_a/(R_s*(R_c + R_a)))
    
    
    T_pre = w_c*PM_c
    LE_pre = w_s*PM_s + w_c*PM_c
    
    return T_pre



In [2]:
#pip install tensorflow --upgrade
from my_functions_upgraded import *
import gc
import mpl_scatter_density
import math
import h5py
import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
print(tf.__version__)
import matplotlib
# matplotlib.use('Agg')
from matplotlib import pyplot as plt
import scipy as sp
from scipy import optimize
from scipy.optimize import fsolve
import pandas as pd
import argparse
import datetime
import matplotlib.dates as mdate
from sklearn.model_selection import train_test_split
import os
import numpy.ma as ma
import pickle
#---- Basic settings ----
sites = os.listdir('../../Input_Data/FluxData/')
sites = [x[0:6] for x in sites]
sites = np.unique(sites)
sites.sort()
#sites = sites[1:2]
opt_hyp_params_info = pd.read_csv('Docs/hyperparameters.csv')

var_names_long = ['DateTime','TA_F_MDS','SW_IN_F_MDS','fPAR','VPD_F_MDS','WS','RH','CO2_F_MDS','LAI','NETRAD','G_F_MDS',\
                  'hc','USTAR','GPP_NT_VUT_REF','theta','theta_2','LE_F_MDS','LE_c','H_F_MDS','H_c','QCflag', 'T_ET_TEA',\
                  'T_ET_uWUE','T_ET_uWUE_BC','LE_SW_emp_calib_LE','Tr_SW_emp_calib_LE','LE_SW_emp_calib_with_TEA',\
                  'Tr_SW_emp_calib_with_TEA','LE_SW_emp_calib_with_uWUE','Tr_SW_emp_calib_with_uWUE',\
                  'LE_SW_emp_calib_with_Liuyang','Tr_SW_emp_calib_with_Liuyang',
                  'rss', 'delta','rou', 'Psy', 'Cp', 'ras', 'rac', 'raa', 'T_ET_Liuyang_2022',\
                 'LE_SW_hydr_calib_LE','Tr_SW_hydr_calib_LE',\
                   'LE_SW_hydr_calib_with_TEA','Tr_SW_hydr_calib_with_TEA','LE_SW_hydr_calib_with_uWUE',\
                   'Tr_SW_hydr_calib_with_uWUE','LE_SW_hydr_calib_with_Liuyang','Tr_SW_hydr_calib_with_Liuyang']
var_names_short = ['DateTime','TA','SRad','fPAR','VPD','WS','RH','CO2','LAI','Rn','G','hc','USTAR','GPP','theta','theta_2','LE','LE_c','H',\
                   'H_c','QCflag','T_ET_TEA','T_ET_uWUE','T_ET_uWUE_BC','LE_SW_emp_calib_LE','Tr_SW_emp_calib_LE',\
                   'LE_SW_emp_calib_with_TEA','Tr_SW_emp_calib_with_TEA','LE_SW_emp_calib_with_uWUE','Tr_SW_emp_calib_with_uWUE',\
                   'LE_SW_emp_calib_with_Liuyang','Tr_SW_emp_calib_with_Liuyang',\
                   'rss', 'delta','rou', 'Psy', 'Cp',\
                   'ras','rac', 'raa', 'T_ET_Liuyang_2022','LE_SW_hydr_calib_LE','Tr_SW_hydr_calib_LE',\
                   'LE_SW_hydr_calib_with_TEA','Tr_SW_hydr_calib_with_TEA','LE_SW_hydr_calib_with_uWUE',\
                   'Tr_SW_hydr_calib_with_uWUE','LE_SW_hydr_calib_with_Liuyang','Tr_SW_hydr_calib_with_Liuyang']
X_vars = ['Rn','G','TA','SRad','fPAR','USTAR','VPD','WS','CO2','LAI','theta'] # X variables for ML model
Y_var = ['LE']       # Target for ML
Auxi_Vars = ['LE','LE_c','H','H_c','QCflag','T_ET_TEA','T_ET_uWUE','T_ET_uWUE_BC',\
             'LE_SW_emp_calib_LE','Tr_SW_emp_calib_LE','LE_SW_emp_calib_with_TEA','Tr_SW_emp_calib_with_TEA',\
             'LE_SW_emp_calib_with_uWUE','Tr_SW_emp_calib_with_uWUE',\
             'LE_SW_emp_calib_with_Liuyang','Tr_SW_emp_calib_with_Liuyang',\
             'rss','delta','rou','Psy','Cp','ras','rac','raa','T_ET_Liuyang_2022','Rn','G','LAI','VPD',\
            'LE_SW_hydr_calib_LE','Tr_SW_hydr_calib_LE',\
                   'LE_SW_hydr_calib_with_TEA','Tr_SW_hydr_calib_with_TEA','LE_SW_hydr_calib_with_uWUE',\
                   'Tr_SW_hydr_calib_with_uWUE','LE_SW_hydr_calib_with_Liuyang','Tr_SW_hydr_calib_with_Liuyang'] # Additional variables

for site in sites:
    print("----" + site + "----")
    try:
        #----Import Data (Prepare data for model training and validation----
        X_train_temp = pd.read_csv('../../Input_Data/Train_Val_Data/X_train_temp_'+site+'.csv')
        X_val_temp = pd.read_csv('../../Input_Data/Train_Val_Data/X_val_temp_'+site+'.csv')
        Y_train = pd.read_csv('../../Input_Data/Train_Val_Data/Y_train_'+site+'.csv')
        Y_val = pd.read_csv('../../Input_Data/Train_Val_Data/Y_val_'+site+'.csv')
        
        X_train = X_train_temp[X_vars]
        X_val = X_val_temp[X_vars]
        X_auxi_train = X_train_temp[Auxi_Vars]
        X_auxi_val = X_val_temp[Auxi_Vars]
        # Backup dataframe
        X_train_back = X_train
        X_val_back = X_val
        X_test = X_val
        Y_test = Y_val
        Y_train = np.asarray(Y_train.T)
        Y_val = np.asarray(Y_val.T)
        Y_test = np.asarray(Y_test.T)
        X_train = np.asarray(X_train.T)
        X_val = np.asarray(X_val.T)
        X_test = np.asarray(X_test.T)
        # ----- Normalization of the Input Data ----
        #var_names = {'TA','SRad','VPD','WS','RH','CO2','LAI','Rn','G','GPP','SM'}
        # Normalization
        mean = X_train.T.astype('float').mean(axis=0)
        std = X_train.T.astype('float').std(axis=0)
        X_train = ((X_train.T - mean) / std).T
        X_val = ((X_val.T - mean) / std).T
        X_test  = ((X_test.T - mean) / std).T
        
        X_train[8,:][np.isnan(X_train[8,:])] = 0  # CO2
        X_val[8,:][np.isnan(X_val[8,:])] = 0  # CO2
        X_test[8,:][np.isnan(X_test[8,:])] = 0  # CO2

        X_auxi_train = np.asarray(X_auxi_train.T)
        X_auxi_val = np.asarray(X_auxi_val.T)
        X_auxi_test = X_auxi_val
        print('Data preparation Done!')
        #------------------ ML model training-------------------------------------
        # Basic settings
        learning_rate = 0.001
        minibatch_size = 64
        print_cost = True
        num_epochs = opt_hyp_params_info['epoch'][opt_hyp_params_info['Site']==site].item()
        n_hidden = opt_hyp_params_info['n_hidden'][opt_hyp_params_info['Site']==site].item()
        n_neurons = opt_hyp_params_info['n_neuron'][opt_hyp_params_info['Site']==site].item()

        # Load parameters from saved pickle file
        parameters = pickle.load(open('Trained_parameters/' + site + '_trained_ANN_params.pkl', 'rb'))
        #---- Prediction ----(Training)   
        rsc_pred_train = predict_tuning(X_train,n_hidden, parameters)[0]
        LE_pred_train = SW_model_LE_np(X_auxi_train.astype('float'), rsc_pred_train.astype('float'))
        T_pred_train = SW_model_T_np(X_auxi_train.astype('float'), rsc_pred_train.astype('float'))

        
        
        #---- Prediction ----(Validation)   
        rsc_pred_val = predict_tuning(X_val,n_hidden, parameters)[0]
        LE_pred_val = SW_model_LE_np(X_auxi_val.astype('float'), rsc_pred_val.astype('float'))
        T_pred_val = SW_model_T_np(X_auxi_val.astype('float'), rsc_pred_val.astype('float'))       

        X_train_temp['LE_pred'] = LE_pred_train
        X_val_temp['LE_pred'] = LE_pred_val
        X_train_temp['T_pred'] = T_pred_train
        X_val_temp['T_pred'] = T_pred_val
        X_train_temp['rsc'] = np.exp(rsc_pred_train)
        X_val_temp['rsc'] = np.exp(rsc_pred_val)
        
        X_train_temp['Period'] = "Train"
        X_val_temp['Period'] = "Val"
        
        X_train_temp.to_csv('Predictions/Train_predictions_' + site + '.csv', index=False)
        X_val_temp.to_csv('Predictions/Val_predictions_' + site + '.csv', index=False)
        X_all = pd.concat([X_train_temp, X_val_temp])
        X_all.to_csv('Predictions/All_predictions_' + site + '.csv', index=False)
    except (OSError,ValueError):
        pass
         

2.12.0-rc1
2.12.0-rc1
----.DS_St----
----AR-SLu----
Data preparation Done!
----AT-Neu----
Data preparation Done!
----AU-ASM----
Data preparation Done!
----AU-Cpr----
Data preparation Done!
----AU-Cum----
Data preparation Done!
----AU-DaP----
Data preparation Done!
----AU-DaS----
Data preparation Done!
----AU-Dry----
Data preparation Done!
----AU-Emr----
Data preparation Done!
----AU-GWW----
Data preparation Done!
----AU-Gin----
Data preparation Done!
----AU-How----
Data preparation Done!
----AU-Rig----
Data preparation Done!
----AU-Stp----
Data preparation Done!
----AU-TTE----
Data preparation Done!
----AU-Tum----
Data preparation Done!
----AU-Ync----
Data preparation Done!
----BE-Lon----
Data preparation Done!
----BE-Vie----
Data preparation Done!
----BR-Sa3----
Data preparation Done!
----CA-NS1----
----CA-NS2----
----CA-NS4----
----CA-NS5----
----CA-NS6----
----CA-NS7----
----CA-Qfo----
Data preparation Done!
----CA-SF1----
Data preparation Done!
----CA-SF2----
Data preparation Done!

# 